In [26]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import itertools
from tqdm.auto import tqdm
import tweepy
import re
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#for Chrome users 
from webdriver_manager.chrome import ChromeDriverManager 
# browser= webdriver.Chrome(executable_path=ChromeDriverManager().install())
import json
import datetime
import numpy as np

### 1. Can be rewritten for class Parcer

In [2]:
#twitter api auth

def new_client():
    tweclient = tweepy.Client(
    consumer_key="",
    consumer_secret="",
    bearer_token= "",
    access_token="",
    access_token_secret="",
    )
    return tweclient

client = new_client()


In [56]:
# https://developer.twitter.com/en/docs/twitter-api/v1/rules-and-filtering/search-operators

def tweeter_search_retreiver(generator, limit) -> pd.DataFrame:
    value_container = []
    search_generator = generator
    final_df = pd.DataFrame(itertools.islice(tqdm(search_generator,total=limit,desc = "Tweets pulled"), limit))
    final_df["user_name"] = final_df["user"].apply(lambda x: x["username"])
    final_df["user_id"] = final_df["user"].apply(lambda x: x["id"])
    return  final_df

#sort now disabled

def get_tweets_by_query(query: str, sort: str = 'latest',limit: int = 100,
                        since: str = '2020-01-01', until: str = None):
    # Using TwitterSearchScraper to scrape data and append tweets to list
    if not until:
        until = str(datetime.date.today())
    twi_generator = sntwitter.TwitterSearchScraper(f'{query} since:{since} until:{until}').get_items()
    return tweeter_search_retreiver(twi_generator,limit)

#sort now disabled

def get_tweets_from_user(user_name: str, sort: str = 'latest', limit: int = 100,
                        since: str = None, until: str = None):
    # Using TwitterSearchScraper to scrape data and append tweets to list
    if not since:
        since = str(get_user_info(user_name)["created"].date())
    if not until:
        until = str(datetime.date.today())
        
    twi_generator = sntwitter.TwitterSearchScraper(f'from:{user_name} since:{since} until:{until}').get_items()
    return tweeter_search_retreiver(twi_generator,limit) 

def get_user_subscribers(user_id: str, max_subs:int = 1000) -> pd.DataFrame:
#     response = client.get_users_followers(user_id, max_results=max_results)
    limit = max_subs/1000
    paginator = tweepy.Paginator(client.get_users_followers, user_id,
                                 max_results=1000, limit=limit)
    dicts_list = []
    for response in paginator: 
        dict_res = get_user_subs_paginator_parcer(response)
        dicts_list.append(dict_res)
    
    names = set(dicts_list[0])
    all_res = {k:[] for k in dict(dicts_list[0]).keys()}
    for name in names:
        for i in range(len(dicts_list)):
            all_res[name]+=dicts_list[i][name]
    all_res["user_parent"] = user_id
    return pd.DataFrame(all_res)

def get_user_subs_paginator_parcer(response):
#     response = client.get_users_followers(user_id, max_results=max_results)

    followers_amount = response.meta["result_count"]

    followers_dict = {k:[] for k in dict(response[0][0]).keys()}
    for sub in range(len(response[0])):
        for atrr,value in dict(response[0][sub]).items():
            followers_dict[atrr] +=[value]
    return followers_dict

def get_user_id_by_name(user_name: str) -> dict:
    response = client.get_user(username=user_name)
    return dict(response[0])

def get_user_info(user_name: str) -> dict:
    response = sntwitter.TwitterUserScraper(user_name)._get_entity()
    return response.__dict__

#requires your webdriver
def get_user_folowers_scrapper(authors: list = ["matveymex"],followers_limit: int = 100,
                              credentials: dict = None,phone_cred = False,
                               save_in: str = "followers.csv") -> pd.DataFrame:
    
    browser = webdriver.Chrome('/Users/matveymex/Downloads/chromedriver_mac_arm64/chromedriver')# Optional argument, if not specified will search path.

    if not credentials:
        raise ConnectionError('Add twitter credentials')
        credentials = {
            "user_name": "",
            "password": "",
            "phone_number":""
        }

    def login():
        browser.get("https://twitter.com/i/flow/login")
        time.sleep(5)

        browser.find_element("xpath","//input[@name='text']").click()
        time.sleep(2)

        browser.find_element("xpath","//input[@name='text']").send_keys(credentials["user_name"])
        time.sleep(2)
        browser.find_element("xpath","(//div[@class='css-901oao r-1awozwy r-6koalj r-18u37iz r-16y2uox r-37j5jr r-a023e6 r-b88u0q r-1777fci r-rjixqe r-bcqeeo r-q4m81j r-qvutc0'])[3]").click()
        time.sleep(2)
        
        browser.find_element("xpath","//input[@name='password']").send_keys(credentials["password"])
        time.sleep(5)
        browser.find_element("xpath","(//div[@class='css-901oao r-1awozwy r-6koalj r-18u37iz r-16y2uox r-37j5jr r-a023e6 r-b88u0q r-1777fci r-rjixqe r-bcqeeo r-q4m81j r-qvutc0'])[3]").click()
        time.sleep(5)
        if phone_cred:
            browser.find_element("xpath","//input[@name='text']").send_keys(credentials["phone_number"])
            time.sleep(2)  
            browser.find_element("xpath","(//div[@class='css-18t94o4 css-1dbjc4n r-sdzlij r-1phboty r-rs99b7 r-19yznuf r-64el8z r-1ny4l3l r-1dye5f7 r-o7ynqc r-6416eg r-lrvibr'])").click()
            time.sleep(2)
        pass

    login()
    users=[]
    pd.DataFrame(columns=["Author","Followers"]).to_csv(save_in, index = False)    
    for author in tqdm(authors,desc = f"followers"):
        print(author)
        max_followers_count = get_user_info(author)["followersCount"]
        print(max_followers_count)
        browser.get(f"https://twitter.com/{author}/followers")
        print(f"scrolling {author}")
        i=0
        local_users = []

        while i<10000:
            time.sleep(2)
            browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            html=browser.page_source
            soup=BeautifulSoup(html)
            main_page=soup.find_all('div',class_="css-1dbjc4n r-kemksi r-1kqtdi0 r-1ljd8xs r-13l2t4g r-1phboty r-16y2uox r-1jgb5lz r-11wrixw r-61z16t r-1ye8kvj r-13qz1uu r-184en5c")
            usernames=main_page[0].find_all('div',class_="css-901oao css-1hf3ou5 r-1bwzh9t r-18u37iz r-37j5jr r-1wvb978 r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-qvutc0")
            print(usernames)
            time.sleep(2)
            
            for un_html in usernames:
                user_name = re.search(r'@(.*?)<', str(un_html)).group(0)[:-1]

                if user_name not in [i[1] for i in local_users]:
                    local_users.append([author,user_name])
   
            print(local_users)
            if len(local_users)>=followers_limit:
                print(f"reached followers limit ({followers_limit})")
                break

            if len(local_users)>=max_followers_count:
                print("end of page")
                break

            if i%50==0:
                print(f"page: {i}, ","follower count: ",len(local_users))
            i=i+1
        pd.DataFrame(local_users,columns=["Author","Followers"]).to_csv(save_in,mode = 'a',
                                                                        header=False, index = False)
        users+=local_users


    df=pd.DataFrame(users,columns=["Author","Followers"])
    browser.close()
    return(df)


In [578]:
# tweets search by request
tweets = get_tweets_by_query("crypto", sort = 'latest',limit = 100,
                        since = '2020-01-01', until = '2023-02-24')
tweets.head(2)

Tweets pulled:   0%|          | 0/100 [00:00<?, ?it/s]

,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
0,https://twitter.com/DigFinGroup/status/1628907...,2023-02-23 23:59:59+00:00,Hong Kong's retail crypto rulz - https://t.co/...,Hong Kong's retail crypto rulz - mailchi.mp/di...,1628907556847460352,"{'username': 'DigFinGroup', 'id': 824055407849...",0,0,3,2,...,NaN,None,None,None,None,None,None,None,1666.0,None
1,https://twitter.com/RickiCryptoStix/status/162...,2023-02-23 23:59:58+00:00,@Mythyks @MonkeDAO @SolanaMBS @hadeswap Good t...,@Mythyks @MonkeDAO @SolanaMBS @hadeswap Good t...,1628907551164338177,"{'username': 'RickiCryptoStix', 'id': 14334986...",0,0,1,0,...,1.628906e+18,"{'username': '0xMyths', 'id': 583638929, 'disp...","[{'username': 'MonkeDAO', 'id': 14301815331980...",None,None,None,None,None,44.0,None


In [584]:
# pull user tweets 
# sort now disabled
tweets_user = get_tweets_from_user("matveymex", sort = 'latest',limit = 100,
                        since = '2020-01-01', until = '2023-02-24')
tweets_user.head(2)

Tweets pulled:   0%|          | 0/100 [00:00<?, ?it/s]

,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
0,https://twitter.com/Matveymex/status/145077637...,2021-10-20 10:50:15+00:00,Я: я работаю ...,Я: я работаю ...,1450776371567923200,"{'username': 'Matveymex', 'id': 82683686871723...",0,0,0,0,...,NaN,None,None,None,None,None,None,None,None,None
1,https://twitter.com/Matveymex/status/141211378...,2021-07-05 18:18:57+00:00,"Ок, а где орехи в шишках, лежащих на улице? Я ...","Ок, а где орехи в шишках, лежащих на улице? Я ...",1412113789617586180,"{'username': 'Matveymex', 'id': 82683686871723...",0,0,0,0,...,NaN,None,None,None,None,None,None,None,None,None


In [ ]:
# official api
# get user subscribers by id
subs = get_user_subscribers(user_id = "1554539094986080256", max_subs = 1000)
subs.head(2)

In [591]:
# user id by name
user_id = get_user_id_by_name("matveymex")
user_id

{'id': 826836868717232128, 'name': 'Майоров Матвей', 'username': 'Matveymex'}

In [688]:
# user info by name
user_info = get_user_info("UltimateApp")
user_info

{'username': 'UltimateApp',
 'id': 1450440581168181249,
 'displayname': 'Ultimate | 🥽 | The DeFi Wallet',
 'rawDescription': 'Building the ultimate wallet for trading & DeFi. We’re live on Solana ◎. Ethereum Ξ 🔜 https://t.co/l6dG4k4hg3',
 'renderedDescription': 'Building the ultimate wallet for trading & DeFi. We’re live on Solana ◎. Ethereum Ξ 🔜 ultimate.app',
 'descriptionLinks': [TextLink(text='ultimate.app', url='http://ultimate.app', tcourl='https://t.co/l6dG4k4hg3', indices=(86, 109))],
 'verified': False,
 'created': datetime.datetime(2021, 10, 19, 12, 36, 47, tzinfo=datetime.timezone.utc),
 'followersCount': 8607,
 'friendsCount': 650,
 'statusesCount': 2341,
 'favouritesCount': 6644,
 'listedCount': 104,
 'mediaCount': 343,
 'location': 'Berlin',
 'protected': False,
 'link': TextLink(text='apps.apple.com/app/apple-stor…', url='https://apps.apple.com/app/apple-store/id1629053410?pt=123577345&ct=social&twitter&ultimate&mt=8', tcourl='https://t.co/pjo5YHDdM7', indices=(0, 23)),


In [595]:
# get user followers selenium 
# requires account credentials (email, password, phone number)
followers = get_user_fllowers_scrapper(["matveymex","elonmask"], followers_limit = 20)
followers.head(2)

/var/folders/rw/050pl3_n08b7_xj95vvldl8h0000gn/T/ipykernel_90989/552403167.py:60: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('/Users/matveymex/Downloads/chromedriver_mac_arm64/chromedriver')  # Optional argument, if not specified will search path.


scrolling matveymex
end of page
scrolling elonmask
reached followers limit (20)


,Author,Followers
0,matveymex,@sabeninastya
1,matveymex,@reeedheaaad
